In [ ]:
 ! pip install -U langchain_community tiktoken  langchainhub chromadb  langgraph langchain_groq

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=hf_embeddings,
)


In [7]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGroq(temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In a LLM-powered autonomous agent system, short-term memory is used for in-context learning, while long-term memory allows the agent to retain and recall information over extended periods, often through an external vector store and fast retrieval.


In [ ]:
#retriever node


from langchain import hub
from langchain_core.output_parsers import StrOutputParser
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

retriever_chain=prompt|llm|StrOutputParser()


In [25]:
question=" What is prompt engineering?"
#retrieval node
retrieved_docs=retriever.get_relevant_documents(question)
doc_txt = retrieved_docs[1].page_content
#Grading node means here we check relevance or not

from pydantic import BaseModel,Field
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

class check_relevance(BaseModel):
  binary_score:str=Field(description="generated_data is relevant,'yes',or,'no'")

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0)
llm_grader_relevance=llm.with_structured_output(check_relevance)

system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
relevance_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","Retrieved documents \n\n:{documents} \n\n user question \n\n:{question}")
    ]
)

relevance_check_grader=relevance_prompt|llm_grader_relevance
relevance_check_grader.invoke({"documents":retrieved_docs,"question":question})



check_relevance(binary_score='yes')

In [27]:
#generae node and cheking hallucination

class hallucination_check(BaseModel):
  binary_score:str=Field(description="generated_data is graunded to the fact,'yes',or,'no'")
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
llm_grader_hallucinate=llm.with_structured_output(hallucination_check)
hallucinate_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","generated documents \n\n:{generated_data} \n\n retrieved documents \n\n:{documents}")
    ]
)


genereted_data=rag_chain.invoke({"context": retrieved_docs, "question": question})

hallucinate_check_grader=hallucinate_prompt|llm_grader_hallucinate
hallucinate_check_grader.invoke({"generated_data":genereted_data,"documents":retrieved_docs})

hallucination_check(binary_score='no')

In [28]:
#check the ans is address the question or not
class check_addressing(BaseModel):
  binary_score:str=Field(description="generated answer is addressing the question,'yes',or,'no'")
llm_grader_addressing=llm.with_structured_output(check_addressing)
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""

addressing_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","generated documents \n\n:{generated_data} \n\n question \n\n:{question}")
    ]
)


# genereted_data=rag_chain.invoke({"context": retrieved_docs, "question": question})

addressing_check_grader=addressing_prompt|llm_grader_addressing
addressing_check_grader.invoke({"generated_data":genereted_data,"question":question})

check_addressing(binary_score='yes')

In [29]:
# rewriting the query for better retrieval
system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying sematic intent / meaning. Just say the question. I dont need any explanation just question is enough"""


rewriting_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","  user  question \n\n:{question}")
    ]
)

question_rewriter=rewriting_prompt|llm|StrOutputParser()
question_rewriter.invoke({"question":question})

'What are the techniques used to design and optimize natural language prompts for effective model performance?'

In [30]:
# graph state

from typing_extensions import TypedDict
from typing import List
class GraphState(TypedDict):
  question:str
  generated_data:str
  documents:List[str]



In [66]:
#nodes
def retriever(state):
  print("--Retriever_node---")
  question=state['question']
  documents=state['documents']
  retriever = vectorstore.as_retriever()
  retrieved_docs=retriever.get_relevant_documents(question)
  return {'documents':retrieved_docs,"question":question}


def relevance_check_grade(state):
  documents=state['documents']
  question=state['question']
  score=relevance_check_grader.invoke({"documents":documents,"question":question})
  grade=score.binary_score
  filter_docs=[]
  for doc in documents:
      score=relevance_check_grader.invoke({"documents":doc.page_content,"question":question})
      grade=score.binary_score
      if grade=='yes':
        print("--Grade:__Document_relevant_----")
        filter_docs.append(doc)
      else:
        print("--Grade:__Document_not_relevant__----")
        continue
  return {"documents": filter_docs, "question": question}

def generate(state):
  print("Generate ans grade ")
  documents=state['documents']
  question=state['question']
  generation = rag_chain.invoke({"context": documents, "question": question})
  return {"documents": documents, "question": question, "generation": generation}
def transform_query(state):
  print("--node rewriting the query---")
  question=state['question']
  new_question=question_rewriter.invoke({"question":question})
  return {"question":new_question}
def decide_to_generate(state):
  print("------ASSESS GRADED DOCUMENTS-----")
  question=state['question']
  filtered_documents = state["documents"]
  if not filtered_documents:
    print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
    return "transform_query"
  else:
    print("---DECISION: DOCUMENTS ARE RELEVANT TO QUESTION,GENERATE--")
    return "generate"

# def hallucinate_check_grade(state):




def hallucinate_check_grade(state):
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # Change 'generade_data' to 'generated_data'
    score=hallucinate_check_grader.invoke({"generated_data":generation,"documents":documents})
    grade=score.binary_score
    if grade == 'yes':
      print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
      # Check question-answering
      print("---GRADE GENERATION vs QUESTION---")
      score=addressing_check_grader.invoke({"generated_data":genereted_data,"question":question})
      grade=score.binary_score
      if grade=='yes':
        print("---DECISION: GENERATION ADDRESSES QUESTION---")
        return "usefull"
      else:
        print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
        return "not useful"
    else:
      print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
      return "not supported"










In [67]:
#graph nodes
from langgraph.graph import END, StateGraph
workflow=StateGraph(GraphState)

workflow.add_node("retriever",retriever)
workflow.add_node("generate",generate)
workflow.add_node("relevance_check_grade",relevance_check_grade)
workflow.add_node("transform_query",transform_query)


workflow.set_entry_point("retriever")
workflow.add_edge("retriever",'relevance_check_grade')
workflow.add_conditional_edges(
    "relevance_check_grade",
   decide_to_generate,
    {
    "transform_query":"transform_query",
    "generate":"generate",

}
    )
workflow.add_edge("transform_query","retriever")
workflow.add_conditional_edges(
    "generate",
    hallucinate_check_grade,
     {

    "usefull":END,
    "not useful":"transform_query",
    "not supported":"generate"
})

In [68]:
app=workflow.compile()

In [69]:
from pprint import pprint



In [ ]:
inputs = {"question": "What is Self-Reflection?", "documents": []}  # Add documents key with an empty list

for output in app.stream(inputs):
    for key, value in output.items():

        pprint(f"Node '{key}':")

    pprint("\n---\n")

pprint(value["generation"])